In [ ]:
import pandas as pd
import numpy as np
import fastai
from fastai.vision.all import *

import dill

In [ ]:
fastai.__version__, torch.__version__

In [ ]:
models = [
    '../input/fgvc8hotel/export_res50_HQAdam.pkl', #v3
    '../input/fgvc8hotel/export_dn161_Fa_CE_bs32.pkl', #v7
    '../input/fgvc8hotel/export_dn161_Fa_FL_bs32.pkl', #v8
    '../input/fgvc8hotel/export_res101_Fall_HQAdam.pkl', #v5
    '../input/fgvc8hotel/export_res101_Fall_5it_4.pkl', #v11
    '../input/hotel-train-fastai-densnet161/export_dn161_kaggle_notebook.pkl' # kaggle v2
]

In [ ]:
image_path = '../input/hotel-id-2021-fgvc8/test_images/' 

In [ ]:
submission = pd.read_csv('../input/hotel-id-2021-fgvc8/sample_submission.csv')

test = submission.copy()

test['image'] = '../input/hotel-id-2021-fgvc8/test_images/' +  test['image']
#test['image'] = 'hotel-id-2021-fgvc8/test_images/' + test['image']
test

In [ ]:
probs = None
for model in models:
    learn = load_learner(fname = Path(model), cpu=False, pickle_module=dill)
    test_dl = learn.dls.test_dl(test)
    probs_temp, _ = learn.tta(dl=test_dl, n=4)
    
    if probs == None:
        probs = probs_temp
    else:
        probs += probs_temp


In [ ]:
preds_idx = probs.topk(5)[1]

In [ ]:
preds = [' '.join(map(str, learn.dls.vocab[pred])) for pred in preds_idx]
preds[:5]

In [ ]:
submission['hotel_id'] = preds

submission.to_csv('submission.csv', index = False)

submission.head()